In [183]:
import pandas as pd
import numpy as np

In [241]:
train=pd.read_csv('train.csv',sep='|')
test=pd.read_csv('test.csv',sep='|')

Data Preparation: Add features to the data set

In [242]:
train['totalScanTimeInMinutes'] = train['totalScanTimeInSeconds'] / 60.0
train['scannedLineItems'] = train['scannedLineItemsPerSecond'] * train['totalScanTimeInSeconds']
train['pricePerScannedLineItem'] = train['grandTotal'] / train['scannedLineItems']
train['scansWithoutRegistrationPerScannedLineItem'] = train['scansWithoutRegistration'] / train['scannedLineItems']
train['quantityModificationsPerScannedLineItem'] = train['quantityModifications'] / train['scannedLineItems']
train['lineItemVoidsPerSecond'] = train['lineItemVoids'] / train['totalScanTimeInSeconds']
#rain['scansWithoutRegistrationPerSecond'] = train['scansWithoutRegistration'] / train['totalScanTimeInSeconds']
#train['quantityModificationsPerSecond'] = train['quantityModifications'] / train['totalScanTimeInSeconds']
#train['secondsPerEuro'] = train['totalScanTimeInSeconds'] / train['grandTotal']
#train['lineItemVoidsPerEuro'] = train['lineItemVoids'] / train['grandTotal']
#train['scansWithoutRegistrationPerEuro'] = train['scansWithoutRegistration'] / train['grandTotal']
#train['quantityModificationsPerEuro'] = train['quantityModifications'] / train['grandTotal']

In [243]:
train.corr()

,trustLevel,totalScanTimeInSeconds,grandTotal,lineItemVoids,scansWithoutRegistration,quantityModifications,scannedLineItemsPerSecond,valuePerSecond,lineItemVoidsPerPosition,fraud,totalScanTimeInMinutes,scannedLineItems,pricePerScannedLineItem,scansWithoutRegistrationPerScannedLineItem,quantityModificationsPerScannedLineItem,lineItemVoidsPerSecond
trustLevel,1.000000,0.025264,0.047656,-0.017312,0.017296,0.006506,0.030701,0.027626,0.009241,-0.319765,0.025264,0.006357,0.008270,0.004769,0.001516,0.012482
totalScanTimeInSeconds,0.025264,1.000000,-0.001230,0.019068,0.009066,-0.011415,-0.262988,-0.208450,-0.011623,0.110414,1.000000,0.009769,-0.019434,-0.021238,-0.028598,-0.220757
grandTotal,0.047656,-0.001230,1.000000,0.006321,-0.030573,-0.008163,-0.011885,0.075679,-0.008858,0.001421,-0.001230,-0.006955,0.328316,0.003583,-0.004884,-0.005339
lineItemVoids,-0.017312,0.019068,0.006321,1.000000,-0.017023,-0.007173,-0.004438,0.005114,0.346514,0.063496,0.019068,0.021647,-0.025732,-0.014101,-0.016196,0.094364
scansWithoutRegistration,0.017296,0.009066,-0.030573,-0.017023,1.000000,0.018961,-0.024126,-0.018844,0.002834,0.074123,0.009066,-0.026529,0.002975,0.346238,0.003962,0.009702
quantityModifications,0.006506,-0.011415,-0.008163,-0.007173,0.018961,1.000000,-0.014880,0.010548,0.018318,-0.000864,-0.011415,-0.035571,0.014195,0.022659,0.368689,0.000753
scannedLineItemsPerSecond,0.030701,-0.262988,-0.011885,-0.004438,-0.024126,-0.014880,1.000000,0.692152,-0.056316,-0.023085,-0.262988,0.100724,-0.061587,-0.057469,-0.055892,0.653202
valuePerSecond,0.027626,-0.208450,0.075679,0.005114,-0.018844,0.010548,0.692152,1.000000,0.019886,-0.028873,-0.208450,-0.022396,0.033083,0.038250,0.040377,0.863587
lineItemVoidsPerPosition,0.009241,-0.011623,-0.008858,0.346514,0.002834,0.018318,-0.056316,0.019886,1.000000,-0.090116,-0.011623,-0.519169,0.610629,0.634480,0.616196,0.061248
fraud,-0.319765,0.110414,0.001421,0.063496,0.074123,-0.000864,-0.023085,-0.028873,-0.090116,1.000000,0.110414,0.298423,-0.100765,-0.090615,-0.094393,-0.026932


# Identify and handle multicollinearity

In [244]:
from statsmodels.stats.outliers_influence import variance_inflation_factor    

def calculate_vif_(X, thresh=5):
    variables = list(range(X.shape[1]))
    dropped = True
    while dropped:
        dropped = False
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)
               for ix in range(X.iloc[:, variables].shape[1])]

        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping \'' + X.iloc[:, variables].columns[maxloc] +
                  '\' at index: ' + str(maxloc))
            del variables[maxloc]
            dropped = True

    print('Remaining variables:')
    print(X.columns[variables])
    #return X.iloc[:, variables]

calculate_vif_(train)

dropping 'totalScanTimeInSeconds' at index: 1
dropping 'scannedLineItems' at index: 10
dropping 'grandTotal' at index: 1
Remaining variables:
Index(['trustLevel', 'lineItemVoids', 'scansWithoutRegistration',
       'quantityModifications', 'scannedLineItemsPerSecond', 'valuePerSecond',
       'lineItemVoidsPerPosition', 'fraud', 'totalScanTimeInMinutes',
       'pricePerScannedLineItem', 'scansWithoutRegistrationPerScannedLineItem',
       'quantityModificationsPerScannedLineItem', 'lineItemVoidsPerSecond'],
      dtype='object')


In [258]:
import statsmodels.formula.api as sm
from sklearn.model_selection import train_test_split

y_train= train['fraud']
x_train= train.drop('fraud',axis=1)   
x_train, x_holdout, y_train, y_holdout = train_test_split(x_train, y_train, test_size=0.2, random_state=324)
    
    
model = sm.Logit(y_train, x_train)
result = model.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.015349
         Iterations 21


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                  fraud   No. Observations:                 1503
Model:                          Logit   Df Residuals:                     1489
Method:                           MLE   Df Model:                           13
Date:                Sat, 20 Apr 2019   Pseudo R-squ.:                  0.9255
Time:                        20:18:49   Log-Likelihood:                -23.070
converged:                       True   LL-Null:                       -309.60
                                        LLR p-value:                4.209e-114
==============================================================================================================
                                                 coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------
trustLevel                                   -11.9813      2.637     -4.544      0.000     -17.150      -6.813
totalScanTimeInSeconds                         0.0043      4e+04   1.08e-07      1.000   -7.84e+04    7.84e+04
grandTotal                                     0.1969      0.109      1.813      0.070      -0.016       0.410
lineItemVoids                                  2.4801      0.830      2.989      0.003       0.854       4.106
scansWithoutRegistration                       2.7563      0.936      2.946      0.003       0.923       4.590
quantityModifications                          1.4044      1.303      1.078      0.281      -1.149       3.958
scannedLineItemsPerSecond                     38.1045     16.372      2.327      0.020       6.016      70.194
valuePerSecond                               -42.0435     15.912     -2.642      0.008     -73.231     -10.856
lineItemVoidsPerPosition                     -39.5351     16.902     -2.339      0.019     -72.663      -6.408
totalScanTimeInMinutes                      5.197e-05    2.4e+06   2.16e-11      1.000    -4.7e+06     4.7e+06
scannedLineItems                               0.0666      0.085      0.786      0.432      -0.099       0.233
pricePerScannedLineItem                       -3.3639      2.495     -1.348      0.178      -8.254       1.526
scansWithoutRegistrationPerScannedLineItem   -44.8407     20.051     -2.236      0.025     -84.140      -5.542
quantityModificationsPerScannedLineItem      -28.4901     29.805     -0.956      0.339     -86.908      29.927
lineItemVoidsPerSecond                      -174.5924    129.208     -1.351      0.177    -427.835      78.651
==============================================================================================================

Possibly complete quasi-separation: A fraction 0.90 of observations can be
perfectly predicted. This might indicate that there is complete
quasi-separation. In this case some parameters will not be identified.
"""

In [254]:
from sklearn.metrics import accuracy_score
predictions = round(result.predict(x_holdout))
accuracy_score(y_true = y_holdout, y_pred = predictions)

0.9920212765957447

In [255]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_holdout,predictions)
cm

array([[350,   1],
       [  2,  23]], dtype=int64)

In [256]:
from sklearn.metrics import balanced_accuracy_score
balanced_accuracy_score(y_holdout,predictions)

0.9585754985754986

In [257]:
def get_monetary_value(cm, y_holdout):
        tn, fp, fn, tp = cm.ravel()
        print("True negative: ", tn)
        print("False positive: ", fp)
        print("False negative: ", fn)
        print("True positive: ", tp)
        score = (-25)*fp + (-5)*fn + 5*tp
        print(score, 'for ', len(y_holdout), ' instances in the test set')
        print(score/len(y_holdout), ' per instance in the test set')
        return score
        
monetary_value = get_monetary_value(cm, y_holdout)

True negative:  350
False positive:  1
False negative:  2
True positive:  23
80 for  376  instances in the test set
0.2127659574468085  per instance in the test set
